# Pre-processing Notebook

This notebook outlines the steps required to pre-process the Synthea data for use.

What is given is a standard `output/` synthea folder with all tables included. What is output is a JSON file in the following (simplified) version:

```JSON
[
    {
        "patient_id": "",
        "label": "",
        "demographics": "",
        "encounters": {
            "encounter": [],
            "conditions": [],
            "careplans": []
        }
    }
]
```

## Imports

In [1]:
import pandas as pd
import json

from tqdm import tqdm

from pa_utils import synthea_table_information, remove_text_inside_parentheses

C:\Users\kolton.hauck\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-processing

Take the raw .csv files for each table of interest and: clean, filter

In [5]:
# read in relevant data
synthea_path = "output/csv/"

#### patients ####
patients_df = pd.read_csv(synthea_path + "patients.csv", header=None)
patients_df.columns = synthea_table_information["patients.csv"]["columns"]

#### encounters ####
encounters_df = pd.read_csv(synthea_path + "encounters.csv", header=None)
encounters_df.columns = synthea_table_information["encounters.csv"]["columns"]
encounters_df["Start"] = pd.to_datetime(encounters_df["Start"], utc=True)
encounters_df["Stop"] = pd.to_datetime(encounters_df["Stop"], utc=True)
encounters_df.sort_values(by="Start", inplace=True)
encounters_df.fillna("", inplace=True)
encounters_df["Description"] = encounters_df["Description"].apply(remove_text_inside_parentheses)
encounters_df["ReasonDescription"] = encounters_df["ReasonDescription"].apply(remove_text_inside_parentheses)

#### careplans ####
careplans_df = pd.read_csv(synthea_path + "careplans.csv", header=None, dtype=str)
careplans_df.columns = synthea_table_information["careplans.csv"]["columns"]
careplans_df.drop(columns=["Id", "Start", "Stop", "Patient", "Code", "ReasonCode"], inplace=True)
careplans_df.fillna("", inplace=True)
careplans_df["Description"] = careplans_df["Description"].apply(remove_text_inside_parentheses)
careplans_df["ReasonDescription"] = careplans_df["ReasonDescription"].apply(remove_text_inside_parentheses)

#### conditions ####
conditions_df = pd.read_csv(synthea_path + "conditions.csv", header=None, dtype=str)
conditions_df.columns = synthea_table_information["conditions.csv"]["columns"]
conditions_df["Description"] = conditions_df["Description"].apply(remove_text_inside_parentheses)
conditions_df.drop(columns=["Start", "Stop", "Patient", "Code"], inplace=True)

#### procedures ####
procedures_df = pd.read_csv(synthea_path + "procedures.csv", header=None, dtype=str)
procedures_df.columns = synthea_table_information["procedures.csv"]["columns"]
procedures_df["Start"] = pd.to_datetime(procedures_df["Start"], utc=True)
procedures_df["Stop"] = pd.to_datetime(procedures_df["Stop"], utc=True)
procedures_df.sort_values(by="Start", inplace=True)
procedures_df.fillna("", inplace=True)
procedures_df["Description"] = procedures_df["Description"].apply(remove_text_inside_parentheses)
procedures_df["ReasonDescription"] = procedures_df["ReasonDescription"].apply(remove_text_inside_parentheses)
# drop later -->> # procedures_df.drop(columns=["Start", "Stop", "Patient", "Code", "ReasonCode", "Base_Cost"], inplace=True)

C:\Users\kolton.hauck\AppData\Local\Temp\1\ipykernel_18344\3134239183.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  encounters_df.fillna("", inplace=True)


# Tmp1

Take the filtered csv (df), turn into JSON format -> faster processing

In [18]:
# careplans
grouped_tmp = careplans_df.groupby("Encounter").apply(lambda x: x.drop("Encounter", axis=1).to_dict(orient="records")).to_dict()

with open("data_processed/tmp1/careplans_1.json", "w") as j_file:
    json.dump(grouped_tmp, j_file)

In [25]:
# conditions
grouped_tmp = conditions_df.groupby("Encounter").apply(lambda x: x.drop("Encounter", axis=1).to_dict(orient="records")).to_dict()

with open("data_processed/tmp1/conditions_1.json", "w") as j_file:
    json.dump(grouped_tmp, j_file)

In [ ]:
# procedures
grouped_tmp = procedures_df.drop(columns=[""]).groupby("Encounter").apply(lambda x: x.drop("Encounter", axis=1).to_dict(orient="records")).to_dict()

with open("data_processed/tmp1/procedures_1.json", "w") as j_file:
    json.dump(grouped_tmp, j_file)

In [18]:
# procedures

grouped_tmp = {}

for index, row in tqdm(procedures_df.iterrows(), total=procedures_df.shape[0]):
    curr_id = row["Encounter"]

    curr_dict = {
        "Description": row["Description"],
        "ReasonDescription": row["ReasonDescription"]
    }

    if curr_id in grouped_tmp:
        grouped_tmp[curr_id].append(curr_dict)
    else:
        grouped_tmp[curr_id] = [curr_dict]

with open("data_processed/tmp1/procedures_1.json", "w") as j_file:
    json.dump(grouped_tmp, j_file)

100%|██████████| 7375831/7375831 [09:10<00:00, 13404.46it/s]


In [9]:
# encounters

grouped_tmp = {}

for index, row in tqdm(encounters_df.iterrows(), total=encounters_df.shape[0]):
    grouped_tmp[row["Id"]] = {
        "Description": row["Description"],
        "ReasonDescription": row["ReasonDescription"]
    }

with open("data_processed/tmp1/encounters_1.json", "w") as j_file:
    json.dump(grouped_tmp, j_file)

100%|██████████| 3800643/3800643 [03:46<00:00, 16768.47it/s]


# Tmp 2 - old

In [79]:
"""
1. given single encounter id
2. get encounter level description adn reasondescription
3. get all descriptions for conditions associated with encounter id
4. get all description, reasondescriptions for all careplans assocaited with encounter id
5. get all description, reasondescriptions for all procedures associated with encounter id
"""

def load_enc_dict(encounter_id):
    curr_enc = {}

    encounter_dict = encounters_df[encounters_df.Id == encounter_id].iloc[0].to_dict()
    curr_enc["encounter"] = {
        "Description": encounter_dict["Description"],
        "ReasonDescription": encounter_dict["ReasonDescription"]
    }

    encounter_conditions_df = conditions_df[conditions_df.Encounter == encounter_id]
    curr_enc["conditions"] = encounter_conditions_df[["Description"]].to_dict(orient="records")

    encounter_careplans_df = careplans_df[careplans_df.Encounter == encounter_id]
    curr_enc["careplans"] = encounter_careplans_df[["Description", "ReasonDescription"]].to_dict(orient="records")

    encounter_procedures_df = procedures_df[procedures_df.Encounter == encounter_id]
    curr_enc["procedures"] = encounter_procedures_df[["Description", "ReasonDescription"]].to_dict(orient="records")

    return curr_enc

In [76]:
"""
1. given a single patient id and it's label and date of poi
2. get encounter df
3. if in-class: drop later encounters (past poi)
4. get patient level data
5. append encounter level data
"""

def get_patient_dict(patient_id, label, date_poi):
    patient_dict = patients_df[patients_df.Id == patient_id].iloc[0].to_dict()

    patient_encounters_df = encounters_df[encounters_df.Patient == patient_id]
    
    if date_poi:
        patient_encounters_df = patient_encounters_df[patient_encounters_df["Start"] <= date_poi] # drop encounters after poi

    #@todo1: sort patient_encounters_df

    curr_pat = {
        "patient_id": patient_dict["Id"],
        "label": label,
        "lat": patient_dict["Lat"],
        "lon": patient_dict["Lon"],
        "encounters": []
    }

    encounter_ids = patient_encounters_df.Id.to_list()
    for encounter_id in encounter_ids:
        curr_pat["encounters"].append(load_enc_dict(encounter_id))

    return curr_pat

In [ ]:
"""
1. get all patient ids
2. get the patients with poi
    2.1 retain only the encounters up to the date of poi
    2.2 also get which patients are in class
3. iter over each patient
    3.1 for each patient: get the 'dict' object
"""

patient_ids = patients_df.Id.to_list()[:100] # subset to test

poi = "Plain chest X-ray"
patients_in_class = procedures_df[procedures_df["Description"] == poi][["Patient", "Start"]].drop_duplicates()
procedures_df.drop(columns=["Start", "Stop", "Patient", "Code", "ReasonCode", "Base_Cost"], inplace=True)

idx = patients_in_class.groupby("Patient")["Start"].idxmax() # focus on the last poi given to the patient (eg patient can have multiple chest x-rays )
patients_in_class_last_procedure = patients_in_class.loc[idx]

patients_dicts = []

for patient_id in tqdm(patient_ids):
    label = patient_id in patients_in_class.Patient.to_list()
    date_poi = patients_in_class_last_procedure[patients_in_class_last_procedure["Patient"] == patient_id]

    if date_poi.empty:
        date_poi = None
    else:
        date_poi = date_poi.iloc[0].Start
        print(date_poi)

    patients_dicts.append(get_patient_dict(patient_id, label, date_poi))


# Tmp 2

In [3]:
with open("data_processed/tmp1/careplans_1.json", "r") as j_file:
    careplans_json = json.load(j_file)
    print("loaded careplans")

with open("data_processed/tmp1/conditions_1.json", "r") as j_file:
    conditions_json = json.load(j_file)
    print("loaded conditions")

with open("data_processed/tmp1/encounters_1.json", "r") as j_file:
    encounters_json = json.load(j_file)
    print("loaded encounters")

with open("data_processed/tmp1/procedures_1.json", "r") as j_file:
    procedures_json = json.load(j_file)
    print("loaded procedures")


loaded careplans
loaded conditions
loaded encounters
loaded procedures


In [4]:
synthea_path = "output/csv/"

#### patients ####
patients_df = pd.read_csv(synthea_path + "patients.csv", header=None)
patients_df.columns = synthea_table_information["patients.csv"]["columns"]
print("loaded patients")

#### encounters ####
encounters_df = pd.read_csv(synthea_path + "encounters.csv", header=None)
encounters_df.columns = synthea_table_information["encounters.csv"]["columns"]
encounters_df["Start"] = pd.to_datetime(encounters_df["Start"], utc=True)
encounters_df["Stop"] = pd.to_datetime(encounters_df["Stop"], utc=True)
encounters_df.sort_values(by="Start", inplace=True)
encounters_df.fillna("", inplace=True)
encounters_df["Description"] = encounters_df["Description"].apply(remove_text_inside_parentheses)
encounters_df["ReasonDescription"] = encounters_df["ReasonDescription"].apply(remove_text_inside_parentheses)
print("loaded encounters")

#### procedures ####
procedures_df = pd.read_csv(synthea_path + "procedures.csv", header=None, dtype=str)
procedures_df.columns = synthea_table_information["procedures.csv"]["columns"]
procedures_df["Start"] = pd.to_datetime(procedures_df["Start"], utc=True)
procedures_df["Stop"] = pd.to_datetime(procedures_df["Stop"], utc=True)
procedures_df.sort_values(by="Start", inplace=True)
procedures_df.fillna("", inplace=True)
procedures_df["Description"] = procedures_df["Description"].apply(remove_text_inside_parentheses)
procedures_df["ReasonDescription"] = procedures_df["ReasonDescription"].apply(remove_text_inside_parentheses)
print("loaded procedures")


loaded patients


C:\Users\kolton.hauck\AppData\Local\Temp\1\ipykernel_16356\893348829.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  encounters_df.fillna("", inplace=True)


loaded encounters
loaded procedures


In [5]:
"""
1. given single encounter id
2. get encounter level description adn reasondescription
3. get all descriptions for conditions associated with encounter id
4. get all description, reasondescriptions for all careplans assocaited with encounter id
5. get all description, reasondescriptions for all procedures associated with encounter id
"""

def load_enc_dict(encounter_id):
    curr_enc = {}

    # enc -> description / reasondescription - one
    curr_enc["encounter"] = encounters_json[encounter_id]

    # enc conditions -> description - multiple
    curr_enc["conditions"] = conditions_json.get(encounter_id, [{"Description": ""}])

    # enc careplans -> description / reasondescription - multiple
    curr_enc["careplans"] = careplans_json.get(encounter_id, [{"Description": "", "ReasonDescription": ""}])

    # enc procedures -> description / reasondescription - multiple
    curr_enc["procedures"] = procedures_json.get(encounter_id, [{"Description": "", "ReasonDescription": ""}])

    return curr_enc

In [6]:
"""
1. given a single patient id and it's label and date of poi
2. get encounter df
3. if in-class: drop later encounters (past poi)
4. get patient level data
5. append encounter level data
"""

def get_patient_dict(patient_id, label, date_poi):
    patient_dict = patients_df[patients_df.Id == patient_id].iloc[0].to_dict()

    patient_encounters_df = encounters_df[encounters_df.Patient == patient_id]
    
    if date_poi:
        patient_encounters_df = patient_encounters_df[patient_encounters_df["Start"] <= date_poi] # drop encounters after poi

    #@todo1: sort patient_encounters_df

    curr_pat = {
        "patient_id": patient_dict["Id"],
        "label": label,
        "lat": patient_dict["Lat"],
        "lon": patient_dict["Lon"],
        "encounters": []
    }

    encounter_ids = patient_encounters_df.Id.to_list()
    for encounter_id in encounter_ids:
        curr_pat["encounters"].append(load_enc_dict(encounter_id))

    return curr_pat

In [7]:
"""
1. get all patient ids
2. get the patients with poi
    2.1 retain only the encounters up to the date of poi
    2.2 also get which patients are in class
3. iter over each patient
    3.1 for each patient: get the 'dict' object
"""

patient_ids = patients_df.Id.to_list()

poi = "Plain chest X-ray"
patients_in_class = procedures_df[procedures_df["Description"] == poi][["Patient", "Start"]].drop_duplicates()
#procedures_dropped = procedures_df.drop(columns=["Start", "Stop", "Patient", "Code", "ReasonCode", "Base_Cost"])


In [7]:
# v1
idx = patients_in_class.groupby("Patient")["Start"].idxmax() # focus on the last poi given to the patient (eg patient can have multiple chest x-rays )
patients_in_class_last_procedure = patients_in_class.loc[idx]

patients_dicts = []

for patient_id in tqdm(patient_ids):
    label = patient_id in patients_in_class.Patient.to_list()
    date_poi = patients_in_class_last_procedure[patients_in_class_last_procedure["Patient"] == patient_id]

    if date_poi.empty:
        date_poi = None
    else:
        date_poi = date_poi.iloc[0].Start

    patients_dicts.append(get_patient_dict(patient_id, label, date_poi))

with open("data_processed/json/all.json", "w") as j_file:
    json.dump(patients_dicts, j_file)

 84%|████████▍ | 48443/57728 [3:41:11<1:01:17,  2.52it/s] 

In [9]:
# v2
import pandas as pd
from tqdm import tqdm

# Assuming you have a function get_patient_dict that builds the dictionary for each patient

# Preprocess to avoid repeated operations inside the loop
patient_set = set(patients_in_class["Patient"])

idx = patients_in_class.groupby("Patient")["Start"].idxmax() # focus on the last poi given to the patient (eg patient can have multiple chest x-rays )
patients_in_class_last_procedure = patients_in_class.loc[idx]
last_procedure_dict = patients_in_class_last_procedure.set_index("Patient")["Start"].to_dict()

patients_dicts = []
chunk_size=500

for patient_id in tqdm(patient_ids):
    label = patient_id in patient_set
    date_poi = last_procedure_dict.get(patient_id, None)

    patients_dicts.append(get_patient_dict(patient_id, label, date_poi))

    # Optionally, write to file in chunks to avoid memory overflow
    # if len(patients_dicts) >= chunk_size:
    #     with open("data_processed/json/all.json", "a") as file:
    #         json.dump(patients_dicts, file)
    #     patients_dicts = []  # Reset the list for the next chunk

# Don't forget to write the last chunk
if patients_dicts:
    # with open("data_processed/json/all.json", "a") as file:
    with open("data_processed/json/all.json", "w") as file:
        json.dump(patients_dicts, file)


100%|██████████| 57728/57728 [3:44:42<00:00,  4.28it/s]  


# General DF

This will take the 'all.json' file produced in Tmp 2 step, vectorize all descriptions and get the data into a tabular dataset.

In [11]:
import numpy as np
import json
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm

In [12]:
with open("data_processed/json/all.json", "r") as j_file:
    patients_dict = json.load(j_file)

In [13]:
# generate embeddings

# get set of all descriptions / reasondescriptions
texts = set()

for patient in tqdm(patients_dict):
    for encounter in patient["encounters"]:
        texts.add(encounter["encounter"]["Description"]) # append enc description
        texts.add(encounter["encounter"]["ReasonDescription"]) # append enc reasondescription

        texts = texts | set([_["Description"] for _ in encounter["conditions"]]) # condition desc

        texts = texts | set([_["Description"] for _ in encounter["careplans"]]) # careplan descs

        texts = texts | set([_["ReasonDescription"] for _ in encounter["careplans"]]) # careplan reas descs

        texts = texts | set([_["Description"] for _ in encounter["procedures"]]) # proc descs

        texts = texts | set([_["ReasonDescription"] for _ in encounter["procedures"]]) # proc reas descs

100%|██████████| 57728/57728 [01:37<00:00, 589.13it/s]


In [14]:
def embed_descriptions(descriptions, model_directory, batch_size=32):
    """ Given a dictionary of SNOMED code:description k:v pairs, and an embedding model (IE BERT), returns a dict of description:embedding of each code"""
    tokenizer = BertTokenizer.from_pretrained(model_directory)

    # Load pre-trained model (weights)
    model = BertModel.from_pretrained(model_directory)
    model.eval()  # Put the model in "evaluation" mode, which turns off dropout
    print(f"model loaded | generating embeddings with bs={batch_size}")

    inputs = tokenizer(descriptions, padding=True, truncation=True, return_tensors="pt", max_length=64)
    # Assuming you have enough memory, process the entire list in batches
    embeddings = []

    # Process in batches with tqdm for progress tracking
    for i in tqdm(range(0, len(descriptions), batch_size), desc="Generating Embeddings"):
        batch = inputs[i:i+batch_size]
        with torch.no_grad():
            outputs = model(**batch)

        # Extract pooled output embeddings
        batch_embeddings = outputs.pooler_output
        embeddings.append(batch_embeddings)

    # Concatenate batched embeddings
    embeddings = torch.cat(embeddings, dim=0)

    return {descriptions[i]: embeddings[i] for i in range(len(descriptions))}

In [15]:
# create dict of embeddings
embeddings = embed_descriptions(list(texts), "models/BioLORD-2023/")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MPNetTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type mpnet to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at models/BioLORD-2023/ and are newly initialized: ['embeddings.token_type_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer

model loaded | generating embeddings with bs=32


Generating Embeddings: 100%|██████████| 25/25 [00:28<00:00,  1.14s/it]


In [7]:
patients_dict[0]["encounters"]

[{'encounter': {'Description': 'Well child visit', 'ReasonDescription': ''},
  'conditions': [{'Description': 'Medication review due'}],
  'careplans': [{'Description': '', 'ReasonDescription': ''}],
  'procedures': [{'Description': 'Medication Reconciliation',
    'ReasonDescription': ''}]},
 {'encounter': {'Description': 'Well child visit', 'ReasonDescription': ''},
  'conditions': [{'Description': 'Medication review due'}],
  'careplans': [{'Description': '', 'ReasonDescription': ''}],
  'procedures': [{'Description': '', 'ReasonDescription': ''}]},
 {'encounter': {'Description': 'Well child visit', 'ReasonDescription': ''},
  'conditions': [{'Description': ''}],
  'careplans': [{'Description': '', 'ReasonDescription': ''}],
  'procedures': [{'Description': 'Medication Reconciliation',
    'ReasonDescription': ''}]},
 {'encounter': {'Description': 'Well child visit', 'ReasonDescription': ''},
  'conditions': [{'Description': 'Medication review due'}],
  'careplans': [{'Description':

In [8]:
test = patients_dict[0]["encounters"]

In [9]:
test[0]

{'encounter': {'Description': 'Well child visit', 'ReasonDescription': ''},
 'conditions': [{'Description': 'Medication review due'}],
 'careplans': [{'Description': '', 'ReasonDescription': ''}],
 'procedures': [{'Description': 'Medication Reconciliation',
   'ReasonDescription': ''}]}

In [16]:
def average_enc_feats(list_of_vecs):
    return np.mean(list_of_vecs, axis=0)

In [17]:
def load_encounters(encounters, embeddings, agg_func=average_enc_feats):

    encounter_vecs = []

    for encounter in encounters:
        curr_enc_vec = []

        curr_enc_vec.extend(embeddings[encounter["encounter"]["Description"]]) # append enc description
        curr_enc_vec.extend(embeddings[encounter["encounter"]["ReasonDescription"]]) # append enc reasondescription

        curr_cond_descs_ = [embeddings[_["Description"]] for _ in encounter["conditions"]]
        curr_enc_vec.extend(agg_func(curr_cond_descs_)) # append condition description aggregated

        curr_carep_descs_ = [embeddings[_["Description"]] for _ in encounter["careplans"]]
        curr_enc_vec.extend(agg_func(curr_carep_descs_)) # append careplan description aggregated

        curr_carep_rdescs_ = [embeddings[_["ReasonDescription"]] for _ in encounter["careplans"]]
        curr_enc_vec.extend(agg_func(curr_carep_rdescs_)) # append careplan reasondescription aggregated

        curr_proc_descs_ = [embeddings[_["Description"]] for _ in encounter["procedures"]]
        curr_enc_vec.extend(agg_func(curr_proc_descs_)) # append proc description aggregated

        curr_proc_rdescs_ = [embeddings[_["ReasonDescription"]] for _ in encounter["procedures"]]
        curr_enc_vec.extend(agg_func(curr_proc_rdescs_)) # append proc reasondescription aggregated
        
        encounter_vecs.append(curr_enc_vec)

    return average_enc_feats(encounter_vecs)

In [28]:
try:
    patients_done = pd.read_csv("flattened_dataset.csv", header=None)[0].to_list()
except:
    patients_done = []

with open("flattened_dataset.csv", "a") as txt_file:
    # iter over each patient
    for patient in tqdm(patients_dict):
        if patient["patient_id"] in patients_done:
            continue

        curr_pat = []
        curr_pat.append(patient["patient_id"]) # patient id (index)
        curr_pat.append(int(patient["label"])) # label (signal / target)
        curr_pat.append(patient["lat"]) # lat of patient
        curr_pat.append(patient["lon"]) # lon of patient

        curr_pat.extend(load_encounters(patient["encounters"], embeddings))

        txt_file.write(",".join([str(item) for item in curr_pat]) + "\n")


100%|██████████| 57728/57728 [14:11:00<00:00,  1.13it/s]   


In [29]:
try:
    patients_done = pd.read_csv("flattened_dataset.csv", header=None)[0].to_list()
except:
    patients_done = []

len(patients_done)

57728

In [24]:
pd.read_csv("flattened_dataset.csv", header=None)

,0,1,2,3,4,5,6,7,8,9,...,5370,5371,5372,5373,5374,5375,5376,5377,5378,5379
0,094e2672-7c1b-eba8-3405-20be988a3811,0,40.873811,-109.431528,-0.031033,0.289719,0.027958,-0.227295,-0.150596,0.169259,...,-0.330483,-0.023646,0.022632,-0.018505,0.180497,-0.034652,-0.113333,0.324499,0.288531,-0.020623
1,c6c8fad2-773f-4a81-7737-d4ded883f521,0,40.589988,-111.721005,0.035127,0.249736,-0.092530,-0.167370,-0.073288,0.214831,...,-0.330803,-0.023421,0.022981,-0.019290,0.180746,-0.034797,-0.113931,0.324178,0.289040,-0.020862
2,6e18deb0-7724-65e6-3b62-c052d30cd63e,0,40.697847,-112.205052,-0.027772,0.287580,0.018022,-0.225007,-0.141567,0.172837,...,-0.330576,-0.023856,0.022236,-0.018447,0.179926,-0.034598,-0.113134,0.324633,0.288424,-0.021169
3,3829fbc5-cfb0-341d-b251-27b25ab7cc41,0,40.620239,-111.918419,0.013951,0.262305,-0.105305,-0.211218,-0.131809,0.188591,...,-0.325680,-0.028446,0.028082,-0.013930,0.180542,-0.037471,-0.116612,0.320529,0.285655,-0.016130
4,58460ba5-986e-89eb-3d53-cc3133658f46,0,40.686387,-111.985047,0.047515,0.235343,-0.095788,-0.169949,-0.086596,0.217631,...,-0.328867,-0.025385,0.023512,-0.017824,0.180867,-0.034692,-0.114863,0.322222,0.287333,-0.019064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,394d0493-f050-60fc-80ea-a3dc0afa31fc,0,33.954177,-86.973280,-0.027428,0.285705,0.016409,-0.217754,-0.138904,0.180988,...,-0.327324,-0.027429,0.028610,-0.015873,0.179260,-0.035154,-0.115409,0.322600,0.284266,-0.018300
96,3f4c1c8b-d157-9bbe-64d6-9f1ef6689f80,0,32.496954,-85.362757,-0.035861,0.300091,0.034468,-0.225611,-0.155050,0.164434,...,-0.330576,-0.023856,0.022236,-0.018447,0.179926,-0.034598,-0.113134,0.324632,0.288424,-0.021169
97,e6ff88a7-b53c-84ce-3fe9-2ef77a14e1e6,0,33.548084,-86.992118,0.027519,0.268755,-0.097893,-0.177759,-0.036013,0.192500,...,-0.311539,-0.023682,0.027404,-0.007975,0.184834,-0.032264,-0.117677,0.304834,0.281175,-0.005406
98,24e3419e-2c5f-89e2-d06b-c5d7c62c3cf3,0,33.440866,-88.030875,0.024499,0.266799,-0.094639,-0.164047,-0.038105,0.201948,...,-0.316801,-0.036090,0.025491,-0.018434,0.187608,-0.036414,-0.117916,0.309503,0.279871,-0.013216
